# Install libraries

In [ ]:
#cài đặt thư viện
!pip install -q lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 65.3 MB/s eta 0:00:00


# Collect data

In [ ]:
#download dataset
#!wget https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv
!gdown 1aXs9anuFLEOO2iQ9mUYDOkni04v_Fy9J

Downloading...
From: https://drive.google.com/uc?id=1aXs9anuFLEOO2iQ9mUYDOkni04v_Fy9J
To: /content/iris.csv
100% 3.98k/3.98k [00:00<00:00, 17.5MB/s]


In [ ]:
# Import libraries
import lightning as L
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [ ]:
data = pd.read_csv("/content/iris.csv")
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Iris là dataset nhận dạng loại hoa từ 4 features bao gồm sepal.length, sepal.widt, petal.length, và petal.width

In [ ]:
#encode cho label
labels = {}
for index, element in enumerate(data["variety"].unique()):
  labels[element] = index

data.loc[:, "labels"] = data["variety"].apply(lambda x: labels[x])
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety,labels
0,5.1,3.5,1.4,0.2,Setosa,0
1,4.9,3.0,1.4,0.2,Setosa,0
2,4.7,3.2,1.3,0.2,Setosa,0
3,4.6,3.1,1.5,0.2,Setosa,0
4,5.0,3.6,1.4,0.2,Setosa,0


In [ ]:
#tạo class Dataset để đọc data
columns = data.drop(['variety', "labels"], axis= 1).columns.to_list()

class DataSet(torch.utils.data.Dataset):
  def __init__(self, data, normalizer):
    super(DataSet, self).__init__()
    self.data = data
    self.feature = normalizer.transform(self.data[columns].values)
    self.feature = torch.tensor(self.feature).float()
    self.label = torch.tensor(self.data["labels"].values)

  def __len__(self):
    return len(self.feature)

  def __getitem__(self, idx):
    return {"feature": self.feature[idx],
            "label": self.label[idx]}


In [ ]:
#Chia và chuẩn hóa dataset
from sklearn.preprocessing import StandardScaler
BATCH_SIZE = 128
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
test_data, val_data = train_test_split(test_data, test_size=0.25, random_state=0)
normalizer = StandardScaler()
normalizer.fit(train_data[columns].values)

train_loader = torch.utils.data.DataLoader(DataSet(train_data, normalizer), batch_size=BATCH_SIZE,
                                           shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(DataSet(val_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(DataSet(test_data, normalizer), batch_size=BATCH_SIZE, num_workers=2)

data_loader = next(iter(train_loader))
print(len(data_loader["feature"]))
print(len(data_loader["label"]))
#print(data_loader["feature"], "\n", data_loader["label"])

120
120


In [ ]:
#Tạo model
class Model(L.LightningModule):
  def __init__(self, num_classes=len(labels), learning_rate=5e-3, input_dim=len(columns)):
    super(Model, self).__init__()
    self.learning_rate = learning_rate
    self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, num_classes),
        )

  def forward(self, x):
    return self.mlp(x)

  def training_step(self, batch, batch_idx):
    x = batch["feature"]
    y = batch["label"]
    y_pred = self(x)
    loss = torch.nn.CrossEntropyLoss()(y_pred, y)
    self.log('train_loss', loss, prog_bar=True, on_epoch = True)
    return loss

  def validation_step(self, batch, batch_idx):
    x = batch["feature"]
    y = batch["label"]
    y_pred = self(x)
    val_loss = torch.nn.CrossEntropyLoss()(y_pred, y)
    self.log('val_loss', val_loss, prog_bar=True, on_epoch = True)
    return val_loss

  def test_step(self, batch, batch_idx):
    x = batch["feature"]
    y = batch["label"]
    y_pred = self(x)
    test_loss = torch.nn.CrossEntropyLoss()(y_pred, y)
    self.log('test_loss', test_loss, prog_bar=True, on_epoch = True)
    return test_loss

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    return optimizer

  def predict_step(self, batch, batch_idx, dataloader_idx=0):
    x = batch["feature"]
    y_pred = self(x)
    return y_pred

# Defined model with hyperparameters:
* batch size: 128
* learning rate: 5e-3

In [ ]:
#train model with batch size 128, learning rate 5e-3
model = Model()

## Inital early stopping


trainer = L.Trainer(max_epochs=100, detect_anomaly=True,
                    accelerator="gpu")
trainer.fit(model, train_loader, val_loader)

INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:lightning.pytorch.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type       | P

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
# Test model
trainer.test(model, dataloaders = test_loader)
print(f"-"*100)
## Prediction
y_pred = trainer.predict(model, dataloaders = test_loader)

## Convert y_pred to numpy array
y_pred = np.array([i.cpu().numpy() for i in y_pred]).squeeze()

## Scoring
from sklearn.metrics import classification_report

print("Scoring: \n",classification_report(test_data["labels"], y_pred.argmax(axis=1)))


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.0011871824972331524   │
└───────────────────────────┴───────────────────────────┘

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


----------------------------------------------------------------------------------------------------


Predicting: |          | 0/? [00:00<?, ?it/s]

Scoring: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         4

    accuracy                           1.00        22
   macro avg       1.00      1.00      1.00        22
weighted avg       1.00      1.00      1.00        22



# Defined model with hyperparameters:
* batch size: 256
* learning rate: 5e-4
* epochs: 50

In [ ]:
# Training model with other parameters: batch size 256, learning rate 5e-4, epochs 50

## Batch size for each dataset
BATCH_SIZE = 256
train_loader1 = torch.utils.data.DataLoader(DataSet(train_data, normalizer), batch_size = BATCH_SIZE,
                                           shuffle=True, num_workers=2)
val_loader1 = torch.utils.data.DataLoader(DataSet(val_data, normalizer), batch_size = BATCH_SIZE,
                                           num_workers=2)
test_loader1 = torch.utils.data.DataLoader(DataSet(test_data, normalizer), batch_size = BATCH_SIZE,
                                           num_workers=2)

print('Size of training dataset: ', len(train_loader.dataset))
print(f'-'*20)

## Inital model and training with aforementioned parameters
model1 = Model(learning_rate=5e-4)
trainer1 = L.Trainer(max_epochs=50, detect_anomaly=True, accelerator="gpu")
trainer1.fit(model1, train_loader1, val_loader1)

Size of training dataset:  120
--------------------


INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:lightning.pytorch.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type       | P

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
# Test model
trainer1.test(model1, dataloaders = test_loader1)
print(f"-"*100)
## Prediction
y_pred1 = trainer1.predict(model1, dataloaders = test_loader1)

## Convert y_pred to numpy array
y_pred1 = np.array([i.cpu().numpy() for i in y_pred1]).squeeze()

## Scoring
from sklearn.metrics import classification_report

print("Scoring: \n",classification_report(test_data["labels"], y_pred1.argmax(axis=1)))

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.7945628762245178     │
└───────────────────────────┴───────────────────────────┘

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


----------------------------------------------------------------------------------------------------


Predicting: |          | 0/? [00:00<?, ?it/s]

Scoring: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      0.11      0.20         9
           2       0.33      1.00      0.50         4

    accuracy                           0.64        22
   macro avg       0.78      0.70      0.57        22
weighted avg       0.88      0.64      0.58        22



* Comment:
  
  * Mặc dù model1 training với mini-batch size gấp đôi nhưng step and epoch quá nhỏ nên quá trình training chưa đủ tốt để model hội tụ.